# Render Design with Ansys Pyaedt






## 1. Create the design in Metal
#### 1.1 Preload libraries and classes, then open Metal
Let's start by importing all the key libraries and classes, and opening Metal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, MetalGUI
from qiskit_metal import Dict, Headings

In [3]:
from qiskit_metal.renderers.renderer_ansys_pyaedt.hfss_renderer_aedt import QHFSSPyaedt
QHFSSPyaedt.default_options

{'begin_disable_autosave': True,
 'close_enable_autosave': True,
 'x_buffer_width_mm': 0.25,
 'y_buffer_width_mm': 0.25,
 'sample_holder_top': '1.2mm',
 'sample_holder_bottom': '-1.2mm',
 'Lj': 1e-08,
 'Cj': 0}

In [4]:
design = designs.MultiPlanar()
gui = MetalGUI(design)

10:05PM 54s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png

10:05PM 54s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Could not create pixmap from c:\usershris\qiskit-metal\qiskit_metal\_gui\styles\metal_dark\rc\transparent.png



#### 1.2 Prepare a design consisting of 4 qubits and 4 CPWs
First we import the necessary components and change key global variables to the desired size.

In [5]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [6]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

Let's create the 4-Qubit-4-CPWs design in one shot. Feel free to break the cell below into subcells if you would like to more closely follow the layout generation.

In [7]:
# Enable overwrite, so that we can re-un this cell multiple times without incurring object name collisions
design.overwrite_enabled = True

# We now define the same custom options for all the transmons.
# We only list the options we intend to modify with respect to the default values
options = Dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 3 connection_pads
    connection_pads=Dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

# Create the 4 transmons
q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

# Now we do the same for the CPWs
options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

# We define a handy method to shorten the number of line of code necessary to define the 4 CPW
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Create the 4 CPWs
asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '5.7 mm', f'-{asym}um', flip=True)

# Render the entire design to the Qiskit Metal GUI
gui.rebuild()
gui.autoscale()

## 2. Render into Ansys Pyaedt HFSS
The Ansys HFSS renderer was instantiated during the boot of Qiskit Metal. Let's create a short handle to refer to it in the rest of this notebook.

In [8]:
fourq_hfss = design.renderers.aedt_hfss

#### 2.1 Create an Ansys project


In [9]:
fourq_hfss.start()

pyaedt INFO: using existing logger.


INFO:Global:using existing logger.


pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\Ansys Student\v222\Win64


INFO:Global:AEDT installation Path C:\Program Files\AnsysEM\Ansys Student\v222\Win64


pyaedt INFO: Launching AEDT with the gRPC plugin.


...

INFO:Global:Ansoft.ElectronicsDesktop.2022.2SV  version started with process ID 1932.


pyaedt INFO: pyaedt v0.6.46


INFO:Global:pyaedt v0.6.46


pyaedt INFO: Python version 3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 17:59:51) [MSC v.1935 64 bit (AMD64)]


INFO:Global:Python version 3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 17:59:51) [MSC v.1935 64 bit (AMD64)]


#### 2.2 Setup an Ansys HFSS design
You can either create a new design or select and use an old one.

##### Creating a design

To create a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert HFSS Design`.
3. Change the HFSS design to either eigenmode or modal by right-clicking on the HFSSdesign1 that just got created inside your project (left panel) and then selecting: `Solution Type...`.

To create a new design **automatically**, execute one of the following two cells. The first will create a design with eigenmode solution type, the second one will create a design with drivenmodal solution type.

NOTE: The design will be added to the project that was active when the command `fourq_hfss.start()` was executed.
Note: If a design named `HFSSTransmonQubit` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `HFSSTransmonQubit1`, `HFSSTransmonQubit2`, etc.

In [10]:
fourq_hfss.populate_project_and_design()

pyaedt INFO: Project QHFSSPyaedt_project has been created.


INFO:Global:Project QHFSSPyaedt_project has been created.


pyaedt INFO: Added design 'QHFSSPyaedt_design' of type HFSS.


INFO:Global:Added design 'QHFSSPyaedt_design' of type HFSS.


pyaedt INFO: Aedt Objects initialized


...

INFO:Global:Design Loaded


pyaedt INFO: Successfully loaded project materials !


INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


##### Selecting a design previously created

If you created a design with the above commands, it will be automatically selected and linked to this jupyter notebook session. Instead, if you want to use a design that had been created beforehand in the project, you can select it by following one of following three approaches:
* manually activating the design from the Ansys GUI, **before** you run the `fourq_hfss.start()` command. In the Ansys GUI, you will find the list of designs in the leftmost panel, and you can activate them with a double click.
* using method `fourq_hfss.connect_ansys(*with parameters*)`, specifying which design to connect to.
* using method `activate_ansys_design('name')`.

The next two cells exemplify how to use the last method. We will switch between the two designs that we previously created.

NOTE: this method will also create a design with the given name if the design name is not found, to be able to create a new design it will need to be given the solution type, for example: `fourq_hfss.activate_ansys_design("newEigen", 'eigenmode')`. Failure to specify the type will result in an error message asking to specify the the `solution_type` parameter.

#### 2.3 Render some component from the Metal design

Find below several rendering examples. You can choose to only execute one of them if you are just browsing this notebook.

Notice how we explicitly clear the design before re-rendering. Indeed `render_design()` only adds shapes to the Ansys design. Re-rendering the same shapes will cause violations.

In [13]:
fourq_hfss.clean_user_design()
fourq_hfss.render_design()  # entire Metal design.

pyaedt INFO: Project QHFSSPyaedt_project set to active.


INFO:Global:Project QHFSSPyaedt_project set to active.


pyaedt INFO: Aedt Objects initialized


INFO:Global:Aedt Objects initialized


pyaedt INFO: Variable Manager initialized


...

INFO:Global:Design Loaded


pyaedt INFO: Successfully loaded project materials !


INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


In [12]:
fourq_hfss.clean_user_design()
#fourq_hfss.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])  # single qubit with 2 endcaps.

pyaedt INFO: Project QHFSSPyaedt_project set to active.


INFO:Global:Project QHFSSPyaedt_project set to active.


pyaedt INFO: Aedt Objects initialized


INFO:Global:Aedt Objects initialized


pyaedt INFO: Variable Manager initialized


...

INFO:Global:Successfully loaded project materials !


pyaedt INFO: Materials Loaded


INFO:Global:Materials Loaded


pyaedt INFO: Deleted 57 Objects: vacuum_box,a_wire_Q1_shortline,a_wire_sub_Q1_shortline,b_wire_Q1_shortline,b_wire_sub_Q1_shortline,c_wire_Q1_shortline,c_wire_sub_Q1_shortline,a_wire_Q2_shortline,a_wire_sub_Q2_shortline,b_wire_Q2_shortline,b_wire_sub_Q2_shortline...


INFO:Global:Deleted 57 Objects: vacuum_box,a_wire_Q1_shortline,a_wire_sub_Q1_shortline,b_wire_Q1_shortline,b_wire_sub_Q1_shortline,c_wire_Q1_shortline,c_wire_sub_Q1_shortline,a_wire_Q2_shortline,a_wire_sub_Q2_shortline,b_wire_Q2_shortline,b_wire_sub_Q2_shortline...


#### 2.4 Create of select a setup
This section is only needed if you plan to execute an analysis. This topic will be covered in the tutorial notebooks 4.x.

To **select** a setup for the active design, you can use `activate_ansys_setup()`. If the setup exists, the QRenderer will reference the setup, otherwise, will make a new setup with the name give.  If no name given, the default name of "Setup" will be used.

In [14]:
fourq_hfss.activate_ansys_setup('SetupNEW')

AttributeError: 'QHFSSPyaedt' object has no attribute 'activate_ansys_setup'

**Finally** disconnect the Metal renderer from the Ansys session.

NOTE: If you do not disconnect explicitly, you might not be able to close the Ansys GUI later.

In [16]:
fourq_hfss.stop()
#fourq_hfss.close()

pyaedt INFO: Project files removed from handlers.


INFO:Global:Project files removed from handlers.


True

## 3. Analyze with pyEPR

In [15]:
from qiskit_metal.analyses.quantization import EPRanalysis

In [16]:
eig_qb = EPRanalysis(design, "hfss")

In [17]:
eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1,
 'n_modes': 1,
 'max_delta_f': 0.5,
 'max_passes': 10,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '10 nH', 'Cj': '0 fF'}}

In [18]:
# example: update single setting
eig_qb.sim.setup.max_passes = 6
eig_qb.sim.setup.vars.Lj = '11 nH'
# example: update multiple settings
eig_qb.sim.setup_update(max_delta_f = 0.4, min_freq_ghz = 1.1)

eig_qb.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'min_freq_ghz': 1.1,
 'n_modes': 1,
 'max_delta_f': 0.4,
 'max_passes': 6,
 'min_passes': 1,
 'min_converged': 1,
 'pct_refinement': 30,
 'basis_order': 1,
 'vars': {'Lj': '11 nH', 'Cj': '0 fF'}}

In [19]:
eig_qb.sim.run(name="Qbit", components=['Q1'], open_terminations=[], box_plus_buffer = False)
eig_qb.sim.plot_convergences()

com_error: (-2147221005, 'Invalid class string', None, None)

## References - Miscellaneous pyEPR/Ansys commands
The following commands are for reference only to better understand how the backend code works. They're not meant to be run directly in this notebook as part of the tutorial.

import pyEPR as epr

Connect to Ansys directly from notebook:

pinfo = epr.ProjectInfo(project_path = None, 
                        project_name = None,
                        design_name  = None)
modeler = pinfo.design.modeler

Access methods within HfssDesign class in pyEPR:

epr.ansys.HfssDesign.create_dm_setup
epr.ansys.HfssDesign.create_q3d_setup

Get project and design names:

pinfo.project_name
design._design.GetName()

Filter qgeometry table:

full_table = design.qgeometry.tables['poly']
mask = full_table['subtract'] == False
table = full_table[mask]

Draw centered rectangles:

bigsquare = modeler.draw_rect_center([0, 0, 0], x_size=8, y_size=8, name='bigsquare')
topright = modeler.draw_rect_center([2, 2, 0], x_size=2, y_size=2, name='topright')

Subtracting shapes:

modeler.subtract('bigsquare', ['topright'])

Draw centered box:

modeler.draw_box_center([0, 0, 0], [1, 2, 3])

Draw closed polygon:

trianglepts = [[-1, 5, 0], [1, 5, 0], [0, 7, 0]]
modeler.draw_polyline(trianglepts, closed=True)

Draw polyline:

smallpts = [[2.85, 0, 0], [3.15, 0, 0]]
modeler.draw_polyline(smallpts, closed=False)

Sweep one polyline with another:

modeler._sweep_along_path('Polyline8', 'Polyline7')